In [ ]:
from langchain.tools import tool
from typing import Dict, Any 
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool 
def web_search(query: str) -> Dict[str, Any]:
    """ Search the web for information"""
    return tavily_client.search(query)

In [7]:
system_prompt = """

You are a personal chef. The user will give you a list of ingredients they have left over in their house.

Using the web search tool, search the web for recipes that can be made with the ingredients they have.

Return recipe suggestions and eventually the recipe instructions to the user, if requested.

"""

In [8]:
# Load the model
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.2:1b", 
                   temperature=0,
                   validate_model_on_init=True,
                   seed=42)

In [9]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model,
    tools=[web_search],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

In [10]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="I have some leftover chicken and rice. What can I make?")]},
    config
)

print(response['messages'][-1].content)

Here are some delicious chicken and rice recipe suggestions:

1. **Seriously Good Chicken and Rice Recipe**: A flavorful dish with chicken thighs, garlicky mushrooms, butter rice, and a perfect blend of spices.
2. **One Pot Chicken and Rice**: A simple and comforting recipe made entirely in one pot with fridge and pantry staples.
3. **Oven Baked Chicken and Rice**: A full meal on its own, but also great paired with a simple Middle Eastern salad or yogurt on the side.

Which one would you like to try?


In [11]:
from pprint import pprint
pprint(response)

{'messages': [HumanMessage(content='I have some leftover chicken and rice. What can I make?', additional_kwargs={}, response_metadata={}, id='64eb0d6d-096a-4df2-88a0-be5dbcbb6e45'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-23T05:16:18.1344152Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10730285500, 'load_duration': 9442539800, 'prompt_eval_count': 223, 'prompt_eval_duration': 891978000, 'eval_count': 25, 'eval_duration': 378615200, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b49a2-d765-71f0-a196-5055aa02e52e-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'chicken and rice recipes'}, 'id': '37e6bc43-9872-4300-b878-b1a6f4a3cfe2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 223, 'output_tokens': 25, 'total_tokens': 248}),
              ToolMessage(content='{"query": "chicken and rice recipes", "follow_up_questions": null, "ans